# 02. Data Cleaning | تنظيف البيانات

## 📚 Prerequisites (What You Need First) | المتطلبات الأساسية

**BEFORE starting this notebook**, you should have completed:
- ✅ **Example 1: Data Loading and Exploration** - You need to know how to identify data quality issues first!
- ✅ **Basic pandas knowledge**: DataFrames, indexing, filtering
- ✅ **Understanding of data quality**: What are missing values, duplicates, outliers?

**If you haven't completed these**, you might struggle with:
- Understanding why data cleaning is necessary
- Knowing which cleaning method to use
- Understanding the impact of cleaning on your data

---

## 🔗 Where This Notebook Fits | مكان هذا الدفتر

**This is the SECOND example** - it fixes the problems we found in Example 1!

**Why this example SECOND?**
- **Before** you can preprocess data, you need to clean it
- **Before** you can build models, you need clean data
- **Before** you can make predictions, you need to fix data quality issues

**Builds on**: 
- 📓 Example 1: Data Loading and Exploration (we found the problems, now we fix them!)

**Leads to**: 
- 📓 Example 3: Data Preprocessing (needs clean data to work with)
- 📓 Example 4: Linear Regression (needs clean, preprocessed data)
- 📓 All ML models (all need clean data!)

**Why this order?**
1. Data cleaning fixes **quality issues** (needed before preprocessing)
2. Data cleaning teaches you **when to remove vs. impute** (critical decision-making)
3. Data cleaning shows you **the impact of outliers** (affects model accuracy)

---

## The Story: Cleaning Before Cooking | القصة: التنظيف قبل الطبخ

Imagine you're cooking a meal. **Before** you can cook, you need to clean your ingredients - remove spoiled items, wash vegetables, check for foreign objects. **After** cleaning everything, you can prepare a safe, delicious meal!

Same with machine learning: **Before** building models, we clean our data - remove duplicates, handle missing values, fix outliers. **After** cleaning, we can build accurate, reliable models!

---

## Why Data Cleaning Matters | لماذا يهم تنظيف البيانات؟

Data cleaning is essential for accurate models:
- **Missing Values**: Break ML algorithms - must be handled
- **Duplicates**: Bias your models (same data counted twice)
- **Outliers**: Skew predictions and statistics
- **Wrong Data Types**: Cause errors in calculations
- **Dirty Data = Bad Models**: No amount of ML can fix fundamentally bad data

## Learning Objectives | أهداف التعلم
1. Handle missing values (remove or impute)
2. Remove duplicate rows
3. Detect and handle outliers
4. Convert data types correctly
5. Understand trade-offs between different cleaning methods
6. Know when to remove vs. when to fix data

In [1]:
# Step 1: Import necessary libraries
# These libraries help us clean and analyze data

import pandas as pd  # For data manipulation (cleaning operations)
import numpy as np   # For numerical operations (handling NaN, calculations)
import matplotlib.pyplot as plt  # For visualizations (seeing outliers)
import seaborn as sns  # For statistical plots (data quality visualization)

print("✅ Libraries imported successfully!")
print("\n📚 What each library does:")
print("   - pandas: Clean data (remove, fill, filter)")
print("   - numpy: Handle missing values (NaN operations)")
print("   - matplotlib: Visualize data quality issues")
print("   - seaborn: Create beautiful quality check plots")

## Part 1: Setting the Scene | الجزء الأول: إعداد المشهد

**BEFORE**: We explored our data in Example 1 and found problems - missing values, duplicates, outliers.

**AFTER**: We'll clean the data by fixing all these issues, making it ready for preprocessing and modeling!

**Why this matters**: Dirty data produces unreliable models. Cleaning is non-negotiable for good ML results!

In [ ]:
# Load real-world Titanic dataset with natural data quality issues
# The Titanic dataset is famous for having missing values, which makes it perfect for learning data cleaning!
# This is REAL data from the 1912 Titanic disaster

print("\n📥 Loading Titanic dataset...")
print("تحميل مجموعة بيانات تايتانيك...")

# Load from public URL (well-known dataset)
titanic_url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
df = pd.read_csv(titanic_url)

# pd.read_csv(url)
# - pd.read_csv(): Reads CSV file from URL or local path
# - This is REAL historical data from the Titanic passenger manifest
# - Returns DataFrame with passenger information

print(f"\n✅ Real-world Titanic dataset loaded!")
print(f"   📊 This is REAL data from the 1912 Titanic passenger manifest")
print(f"   📈 Contains {len(df)} passengers with {len(df.columns)} features")
print(f"\n🔍 Notice:")
print("   - This dataset naturally has missing values (Age, Cabin, Embarked)")
print("   - Real data often has data quality issues - this is normal!")
print("   - We'll learn to clean this real data in the following steps")


## 📊 Understanding the Dataset | فهم مجموعة البيانات

### For CS Students - Focus on Data Structure, Not Domain | للطلاب في علوم الحاسوب - ركز على هيكل البيانات، وليس المجال

**As computer science students, you'll work with many different types of datasets** (medical, financial, e-commerce, etc.). **What matters is the data structure, not the domain knowledge!**

**Data Structure Focus**:
- **Data Shape**: ~890 rows × 12 columns (samples × features)
- **Feature Types**: Mixed (numerical: Age, Fare; categorical: Sex, Embarked, Cabin)
- **Target Type**: Classification (predicting survival: 0 or 1)
- **Task**: Predict passenger survival based on features
- **Data Quality**: Has missing values (Age, Cabin, Embarked) - perfect for learning cleaning!

**Why This Structure Matters**:
- **Mixed feature types** → Need different cleaning methods (numeric vs. categorical)
- **Missing values present** → Real-world scenario (most datasets have missing values)
- **Classification task** → We'll use classification metrics later (not regression)
- **Historical data** → Natural data quality issues (real-world scenario)

### Understanding the Dataset Domain (Brief) | فهم مجال مجموعة البيانات (باختصار)

**What is this data?** Titanic passenger manifest from 1912 disaster.

**Why does this matter?** 
- **For choosing cleaning methods**: Missing Age → use mean/median (numeric), Missing Embarked → use mode (categorical)
- **For outlier detection**: Age should be 0-120, Fare should be positive
- **For data types**: Age should be numeric (not text), Sex should be categorical

**Domain Context** (Brief):
- **Survived**: Target variable (0 = died, 1 = survived)
- **Pclass**: Passenger class (1st, 2nd, 3rd) - affects survival chances
- **Sex**: Gender (male/female) - affected survival (women and children first)
- **Age**: Age in years - missing for many passengers
- **Fare**: Ticket price - higher fare might indicate better survival chances
- **Cabin**: Cabin number - many missing (not all passengers had cabins)

**💡 Key Point for CS Students**: You don't need to be a history expert! Focus on:
- Understanding the **data structure** (rows, columns, types, missing values)
- Knowing the **cleaning methods** (remove vs. impute, mean vs. median vs. mode)
- Choosing the right **cleaning approach** based on structure, not domain knowledge


## Step 1: Load Real-World Data with Issues | الخطوة 1: تحميل بيانات العالم الحقيقي بمشاكل

**BEFORE**: We need to learn cleaning techniques, but we need data with real-world problems to practice on.

**AFTER**: We'll load the Titanic dataset - a famous real-world dataset that naturally has missing values, making it perfect for learning data cleaning!

**Why use Titanic?** This is REAL historical data from 1912 with natural data quality issues. Real datasets have these problems! We need to learn how to handle them.

In [ ]:
# Check for duplicate rows in the real data
# Real datasets sometimes have duplicates from data entry errors or merging issues

# First, let's see if there are any natural duplicates
initial_count = len(df)
duplicate_count = df.duplicated().sum()

print(f"\n📊 Checking for duplicates...")
print(f"   Initial rows: {initial_count}")
print(f"   Duplicate rows found: {duplicate_count}")

# For demonstration, we'll add a few duplicates to show the cleaning process
# (In real projects, you'd check if duplicates should be removed or kept)
if duplicate_count == 0:
    # Add 2 duplicates for demonstration purposes
    df = pd.concat([df, df.iloc[[0, 1]]], ignore_index=True)
    print("   - Added 2 duplicate rows for demonstration")
else:
    print("   - Dataset already contains duplicates (real-world scenario!)")

In [ ]:
# Check for outliers in the real data
# Real datasets often have outliers from errors (typos, measurement mistakes) or rare events

# df.shape
# - Returns tuple (rows, columns): (number_of_rows, number_of_columns)
print("\n📊 Original Data Shape:", df.shape)
print("شكل البيانات الأصلية:", df.shape)

# Check for potential outliers in Age (should be reasonable, e.g., 0-120)
if 'Age' in df.columns:
    age_outliers = df[(df['Age'] > 100) | (df['Age'] < 0)]
    print(f"\n🔍 Checking for age outliers...")
    print(f"   Age range: {df['Age'].min():.1f} to {df['Age'].max():.1f}")
    if len(age_outliers) > 0:
        print(f"   Found {len(age_outliers)} potential age outliers")
    else:
        print("   No obvious age outliers found")

# For demonstration, we'll add one impossible age value to show outlier handling
# (In real projects, check if outliers are errors or valid rare cases)
if 'Age' in df.columns and df['Age'].max() < 120:
    df.loc[0, 'Age'] = 150  # Add impossible age for demonstration
    print("   - Added 1 impossible age value (150) for demonstration")

# df.head(10)
# - Returns first 10 rows of DataFrame
# - head(n): Shows first n rows (default is 5)
# - Useful for quick data inspection
print("\n📄 Original Data (first 10 rows):")
print(df.head(10))
print(f"\n📋 Columns: {list(df.columns)}")

In [5]:
# First, let's see how many missing values we have
print("\n" + "=" * 60)
print("2. Handling Missing Values")
print("معالجة القيم المفقودة")
print("=" * 60)

print("\n🔍 Missing values before cleaning:")
print("القيم المفقودة قبل التنظيف:")

# df.isnull().sum()
# - df.isnull(): Returns DataFrame with True/False (True = missing value, False = not missing)
# - .sum(): Sums True values (counts missing values) for each column
# - Returns Series with column names and count of missing values
# - Alternative: df.isna() does the same thing
missing_before = df.isnull().sum()
print(missing_before)

# missing_before.sum()
# - .sum() on Series: Adds up all values in the Series
# - This gives total missing values across all columns
print(f"\n   Total missing values: {missing_before.sum()}")

# df.shape[0] * df.shape[1]
# - df.shape[0]: Number of rows
# - df.shape[1]: Number of columns
# - shape[0] * shape[1]: Total number of cells in DataFrame
# - Used to calculate percentage of missing values
print(f"   Percentage missing: {(missing_before.sum() / (df.shape[0] * df.shape[1]) * 100):.1f}%")


In [6]:
# Method 1: Remove rows with missing values
# .dropna() removes any row that has at least one missing value
# Why use this? If missing values are rare, it's better to remove than guess

print("\n--- Method 1: Remove rows with missing values ---")
print("--- الطريقة 1: حذف الصفوف ذات القيم المفقودة ---")

# df.dropna()
# - Removes rows that contain ANY missing values (NaN/None)
# - Returns new DataFrame (doesn't modify original unless inplace=True)
# - Parameters:
#   - axis=0 (default): Drop rows (axis=1 would drop columns)
#   - how='any' (default): Drop if ANY value is missing
#   - subset=None: Check all columns (can specify columns to check)
#   - inplace=False: Return new DataFrame (True modifies original)
df_removed = df.dropna()

# df.shape[0] - df_removed.shape[0]
# - df.shape[0]: Original number of rows
# - df_removed.shape[0]: Number of rows after removal
# - Difference = number of rows removed
rows_removed = df.shape[0] - df_removed.shape[0]
print(f"✅ Rows after removal: {df_removed.shape[0]} (removed {rows_removed} rows)")
print(f"الصفوف بعد الحذف: {df_removed.shape[0]} (تم حذف {rows_removed} صف)")
print(f"   Data loss: {(rows_removed / df.shape[0] * 100):.1f}%")


## Step 2: Handling Missing Values | الخطوة 2: معالجة القيم المفقودة

**BEFORE**: We have missing values (NaN/None) that will break our ML models.

**AFTER**: We'll either remove rows with missing values OR fill them with reasonable estimates!

**Why handle missing values?** 
- ML algorithms cannot work with missing data
- Missing values indicate incomplete information
- We must decide: **Remove** (if few missing) or **Impute** (if many missing)

**Two main strategies:**
1. **Remove**: Drop rows/columns with missing values (good if <5% missing)
2. **Impute**: Fill missing values with mean/median/mode (good if >5% missing)

In [7]:
# Method 2: Fill missing values (imputation)
# We'll use a copy so we don't modify the original
print("\n--- Method 2: Fill missing values ---")
print("--- الطريقة 2: ملء القيم المفقودة ---")

# df.copy()
# - Creates a deep copy of DataFrame (independent copy, not a reference)
# - Changes to copy don't affect original
# - Important: Without copy(), both variables point to same data
# - Alternative: df.copy(deep=True) is explicit (deep=True is default)
df_filled = df.copy()


In [8]:
# Fill numeric columns with mean
# ملء الأعمدة الرقمية بالمتوسط

# df_filled['Age'].fillna(df_filled['Age'].mean(), inplace=True)
# - df_filled['Age']: Selects 'age' column (returns Series)
# - .fillna(): Fills missing values (NaN) with specified value
#   - Parameter: Value to fill (here: mean of age column)
#   - inplace=True: Modifies DataFrame directly (False returns new Series)
# - df_filled['Age'].mean(): Calculates mean (average) of age column
#   - .mean(): Returns average of all non-missing values
#   - Ignores NaN values automatically
# Result: All missing ages replaced with average age
df_filled['Age'].fillna(df_filled['Age'].mean(), inplace=True)

In [9]:
# Check for and remove duplicate rows
# .duplicated() finds rows that are exact duplicates
# .drop_duplicates() removes them, keeping the first occurrence

print("\n" + "=" * 60)
print("3. Removing Duplicates")
print("إزالة التكرارات")
print("=" * 60)

# df_filled.duplicated().sum()
# - df_filled.duplicated(): Returns boolean Series (True = duplicate row, False = unique)
#   - Checks if each row is identical to a previous row
#   - First occurrence marked as False, duplicates as True
#   - Parameters:
#     - subset=None: Check all columns (can specify columns to check)
#     - keep='first' (default): Mark first as False, rest as True
# - .sum(): Counts True values (number of duplicate rows)
num_duplicates = df_filled.duplicated().sum()
print(f"\n🔍 Number of duplicates: {num_duplicates}")
print(f"عدد التكرارات: {num_duplicates}")

# df_filled.drop_duplicates()
# - Removes duplicate rows, keeps first occurrence
# - Returns new DataFrame (doesn't modify original unless inplace=True)
# - Parameters:
#   - subset=None: Check all columns for duplicates
#   - keep='first' (default): Keep first, remove rest ('last' keeps last, False removes all)
#   - inplace=False: Return new DataFrame
df_no_duplicates = df_filled.drop_duplicates()
print(f"\n✅ Rows after removing duplicates: {df_no_duplicates.shape[0]}")
print(f"الصفوف بعد إزالة التكرارات: {df_no_duplicates.shape[0]}")
print(f"   Removed {num_duplicates} duplicate row(s)")


In [10]:
# Fill categorical columns with mode
# ملء الأعمدة الفئوية بالقيمة الأكثر تكراراً

# df_filled['Embarked'].fillna(df_filled['Embarked'].mode()[0], inplace=True)
# - df_filled['Embarked']: Selects 'department' column
# - .mode(): Returns Series with most frequent value(s)
#   - Mode = most common value (for categorical data)
#   - Returns Series (can have multiple modes if tie)
# - [0]: Gets first mode value (if multiple modes, takes first)
# - .fillna(): Fills missing values with mode
# - inplace=True: Modifies DataFrame directly
# Result: All missing departments replaced with most common department
df_filled['Embarked'].fillna(df_filled['Embarked'].mode()[0], inplace=True)
print("\nMissing values after filling:")
print("القيم المفقودة بعد الملء:")
print(df_filled.isnull().sum())

In [11]:
# IQR (Interquartile Range) Method for outlier detection
# This is a statistical method that identifies values far from the median

print("\n--- IQR Method for Outlier Detection ---")
print("--- طريقة المدى الربيعي للكشف عن القيم الشاذة ---")

def detect_outliers_iqr(series):
    """
    Detect outliers using IQR method.
    Returns True for outliers, False for normal values.
    """
    # series.quantile(0.25)
    # - Calculates 25th percentile (Q1) - value below which 25% of data falls
    # - quantile(q): Returns value at quantile q (0.0 to 1.0)
    # - 0.25 = 25th percentile, 0.5 = median, 0.75 = 75th percentile
    Q1 = series.quantile(0.25)  # 25th percentile
    
    # series.quantile(0.75)
    # - Calculates 75th percentile (Q3) - value below which 75% of data falls
    Q3 = series.quantile(0.75)  # 75th percentile
    
    # IQR = Q3 - Q1
    # - Interquartile Range: Spread of middle 50% of data
    # - Measures variability, less sensitive to outliers than range
    IQR = Q3 - Q1  # Interquartile Range
    
    # Q1 - 1.5 * IQR
    # - Lower fence: Values below this are considered outliers
    # - 1.5 is standard multiplier (can be adjusted)
    lower_bound = Q1 - 1.5 * IQR  # Lower fence
    
    # Q3 + 1.5 * IQR
    # - Upper fence: Values above this are considered outliers
    upper_bound = Q3 + 1.5 * IQR  # Upper fence
    
    # (series < lower_bound) | (series > upper_bound)
    # - Boolean indexing: Creates boolean Series (True = outlier, False = normal)
    # - | : Logical OR operator (element-wise)
    # - Returns True for values outside [lower_bound, upper_bound]
    # Values outside the fences are outliers
    return (series < lower_bound) | (series > upper_bound)

print("   ✅ IQR function defined")
print("   This will identify values that are too far from the median")


In [12]:
# 4. Handling Outliers
# معالجة القيم الشاذة
print("\n" + "=" * 60)
print("4. Handling Outliers")
print("معالجة القيم الشاذة")
print("=" * 60)

In [13]:
# Check for outliers in salary
print("\nOutliers in salary column:")
print("القيم الشاذة في عمود الراتب:")

# detect_outliers_iqr(df_no_duplicates['Fare'])
# - Calls function defined earlier
# - df_no_duplicates['Fare']: Passes salary column as Series
# - Returns boolean Series (True = outlier, False = normal)
fare_outliers = detect_outliers_iqr(df_no_duplicates['Fare'])

# fare_outliers.sum()
# - Counts True values (number of outliers)
print(f"Number of outliers: {fare_outliers.sum()}")

# df_no_duplicates[fare_outliers][['name', 'salary']]
# - df_no_duplicates[fare_outliers]: Boolean indexing - selects rows where fare_outliers is True
#   - Boolean Series used as filter: True rows kept, False rows removed
# - [['name', 'salary']]: Selects only 'name' and 'salary' columns
#   - Double brackets [[]] returns DataFrame (single [] returns Series)
# Result: Shows only outlier rows with name and salary columns
print(df_no_duplicates[fare_outliers][['name', 'salary']])

In [14]:
# Remove outliers

# df_no_duplicates[~salary_outliers].copy()
# - ~salary_outliers: NOT operator (~) inverts boolean Series
#   - True becomes False, False becomes True
#   - Selects rows that are NOT outliers (keeps normal values)
# - [~salary_outliers]: Boolean indexing - filters DataFrame
# - .copy(): Creates independent copy (good practice)
# Result: DataFrame with outliers removed
df_clean = df_no_duplicates[~salary_outliers].copy()

In [15]:
# Also remove impossible age values

# df_clean[df_clean['age'] <= 100].copy()
# - df_clean['age'] <= 100: Creates boolean Series (True if age <= 100, False otherwise)
#   - Comparison operator (<=) applied element-wise to all values
# - df_clean[boolean_series]: Boolean indexing - keeps rows where condition is True
# - .copy(): Creates independent copy
# Result: Keeps only rows where age is 100 or less (removes impossible ages like 150)
df_clean = df_clean[df_clean['age'] <= 100].copy()
print(f"\nRows after removing outliers: {df_clean.shape[0]}")
print(f"الصفوف بعد إزالة القيم الشاذة: {df_clean.shape[0]}")

In [16]:
# 5. Data Type Conversion
# تحويل أنواع البيانات
print("\n" + "=" * 60)
print("5. Data Type Conversion")
print("تحويل أنواع البيانات")
print("=" * 60)
print("\nData types before conversion:")
print("أنواع البيانات قبل التحويل:")

# df_clean.dtypes
# - Returns Series showing data type of each column
# - Common types: int64, float64, object (string), bool, datetime64
# - Useful for checking if types are correct (e.g., numbers stored as strings)
print(df_clean.dtypes)

In [17]:
# Convert experience_years to int (rounding)

# df_clean['experience_years'].round().astype(int)
# - df_clean['experience_years']: Selects 'experience_years' column
# - .round(): Rounds decimal values to nearest integer
#   - No parameter = rounds to 0 decimal places
#   - .round(2) would round to 2 decimal places
# - .astype(int): Converts data type to integer
#   - astype(): Changes data type of Series
#   - int: Integer type (int64 in pandas)
#   - Alternative: .astype('int64') or .astype(float) for float
# Result: Converts float values like 2.5 to integer 3 (after rounding)
df_clean['experience_years'] = df_clean['experience_years'].round().astype(int)

In [18]:
# Final summary of cleaning process
print("\n" + "=" * 60)
print("6. Cleaning Summary")
print("ملخص التنظيف")
print("=" * 60)

original_rows = df.shape[0]
final_rows = df_clean.shape[0]
rows_removed = original_rows - final_rows

print(f"\n📊 Original rows: {original_rows}")
print(f"الصفوف الأصلية: {original_rows}")
print(f"✅ Final cleaned rows: {final_rows}")
print(f"الصفوف النهائية بعد التنظيف: {final_rows}")
print(f"\n🗑️  Rows removed: {rows_removed} ({(rows_removed/original_rows*100):.1f}%)")
print(f"الصفوف المحذوفة: {rows_removed}")

print("\n📄 Cleaned Data (first 10 rows):")
print("البيانات النظيفة:")
print(df_clean.head(10))

print("\n" + "=" * 60)
print("✅ Example 2 Complete! ✓")
print("اكتمل المثال 2! ✓")
print("=" * 60)
print("\n🎓 What you accomplished:")
print("   ✅ Handled missing values (imputed with mean/mode)")
print("   ✅ Removed duplicate rows")
print("   ✅ Detected and removed outliers")
print("   ✅ Converted data types correctly")
print("   ✅ Created clean, model-ready data!")


## Step 6: Decision Framework - When to Remove vs. When to Fix | الخطوة 6: إطار القرار - متى نحذف ومتى نصلح

**BEFORE**: You've learned different cleaning methods, but when should you use each one?

**AFTER**: You'll have a clear decision framework to choose the right cleaning method for any situation!

**Why this matters**: Making the wrong cleaning decision can:
- **Remove too much data** → Lose valuable information
- **Keep bad data** → Break your models
- **Use wrong method** → Introduce bias or errors

---

### 🎯 Decision Framework for Missing Values | إطار القرار للقيم المفقودة

**Key Question**: Should I **REMOVE** or **IMPUTE** missing values?

#### Decision Tree:

```
Is missing data < 5% of total?
├─ YES → REMOVE (dropna)
│   └─ Why? Small loss, keeps data "pure"
│
└─ NO → Is missing data random or systematic?
    ├─ RANDOM → IMPUTE (fillna with mean/median/mode)
    │   └─ Why? Random missing = no bias, safe to estimate
    │
    └─ SYSTEMATIC → INVESTIGATE FIRST
        └─ Why? Systematic missing might indicate important pattern
```

#### Comparison Table:

| Method | When to Use | Pros | Cons | Example |
|--------|-------------|------|------|---------|
| **Remove** | < 5% missing, random | • No bias introduced<br>• Keeps data "pure"<br>• Simple | • Loses data<br>• Can't use if many missing | Age missing in 2% of records |
| **Impute (Mean/Median)** | > 5% missing, numeric, random | • Keeps all rows<br>• Preserves sample size<br>• Works for numeric | • Can introduce bias<br>• Assumes normal distribution | Salary missing in 15% of records |
| **Impute (Mode)** | > 5% missing, categorical, random | • Keeps all rows<br>• Preserves sample size<br>• Works for categories | • Can create artificial patterns<br>• May over-represent common values | Department missing in 10% of records |
| **Investigate** | Systematic missing | • Finds root cause<br>• Prevents bias<br>• Better decisions | • Takes time<br>• Requires domain knowledge | All salaries missing for one department |

---

### 🎯 Decision Framework for Outliers | إطار القرار للقيم الشاذة

**Key Question**: Should I **REMOVE** or **KEEP** outliers?

#### Decision Tree:

```
Is the outlier a data entry error?
├─ YES → REMOVE
│   └─ Example: Age = 150, Salary = 500000 (typo)
│
└─ NO → Is the outlier a rare but valid event?
    ├─ YES → KEEP (but handle separately)
    │   └─ Example: CEO salary in employee dataset
    │
    └─ NO → Does it affect model performance?
        ├─ YES → REMOVE or TRANSFORM
        │   └─ Example: Extreme values breaking linear regression
        │
        └─ NO → KEEP
            └─ Example: Outlier in non-critical feature
```

#### Comparison Table:

| Method | When to Use | Pros | Cons | Example |
|--------|-------------|------|------|---------|
| **Remove** | Data entry errors, impossible values | • Removes noise<br>• Improves model accuracy<br>• Simple | • Loses information<br>• May remove valid rare events | Age = 150, Salary = 500000 |
| **Cap/Clip** | Valid but extreme values | • Keeps data<br>• Reduces impact<br>• Preserves distribution | • Arbitrary threshold<br>• May hide important patterns | Cap salary at 99th percentile |
| **Transform** | Skewed distributions | • Normalizes data<br>• Keeps all values<br>• Better for ML | • Changes interpretation<br>• More complex | Log transform for income |
| **Keep** | Rare but valid events | • Preserves reality<br>• No information loss<br>• Important for analysis | • Can skew models<br>• May need special handling | CEO in employee dataset |

---

### 🎯 Decision Framework for Duplicates | إطار القرار للتكرارات

**Key Question**: Should I **REMOVE** all duplicates or **INVESTIGATE** first?

#### Decision Tree:

```
Are duplicates exact copies?
├─ YES → REMOVE (keep first)
│   └─ Why? No new information, wastes space
│
└─ NO → Are duplicates near-duplicates (typos)?
    ├─ YES → FIX (merge or correct)
    │   └─ Why? Same entity, different spelling
    │
    └─ NO → Are duplicates valid (same person, different records)?
        └─ YES → KEEP (but flag for analysis)
            └─ Why? Important information (e.g., repeat customers)
```

---

### 📊 Real-World Examples | أمثلة من العالم الحقيقي

#### Example 1: E-commerce Dataset
- **Missing values in "price"**: 20% missing
  - **Decision**: IMPUTE with median (too much to remove, random missing)
  - **Reason**: Random missing prices, median preserves distribution

#### Example 2: Medical Dataset
- **Outlier in "age"**: One patient age = 200
  - **Decision**: REMOVE (impossible value)
  - **Reason**: Data entry error, no one lives to 200

#### Example 3: Customer Dataset
- **Missing values in "email"**: 3% missing
  - **Decision**: REMOVE (small percentage)
  - **Reason**: Email is critical, can't impute, small loss acceptable

#### Example 4: Sales Dataset
- **Outlier in "revenue"**: One sale = $1,000,000 (normal range: $10-$1000)
  - **Decision**: INVESTIGATE FIRST
  - **Reason**: Could be valid (enterprise sale) or error (extra zero)

---

### ✅ Key Takeaways | النقاط الرئيسية

1. **Always investigate first** - Understand WHY data is missing/outlier/duplicate
2. **Consider data loss** - Removing >10% of data is usually too much
3. **Consider bias** - Systematic missing/outliers may indicate important patterns
4. **Test both approaches** - Sometimes try removing AND imputing, compare results
5. **Document decisions** - Write down WHY you chose each method (for reproducibility)

---

### 🎓 Practice Decision-Making | ممارسة اتخاذ القرار

**Scenario**: You have a dataset with:
- 8% missing values in "income" column
- 2 outliers in "age" (ages 0 and 200)
- 5 duplicate rows

**Your task**: Decide what to do for each issue and explain why!

**Answers**:
1. **Income (8% missing)**: IMPUTE with median (too much to remove, likely random)
2. **Age outliers**: REMOVE (impossible values - data entry errors)
3. **Duplicates**: REMOVE (exact copies, no new information)


In [19]:
# Practical Example: Comparing Different Cleaning Approaches
# مثال عملي: مقارنة طرق التنظيف المختلفة

print("=" * 60)
print("Practical Example: Decision-Making in Action")
print("مثال عملي: اتخاذ القرار في الممارسة")
print("=" * 60)

# Create a scenario with different data quality issues
print("\n📊 Scenario: Dataset with multiple issues")
print("   - 15% missing values in 'income' (too much to remove)")
print("   - 2% missing values in 'email' (can remove)")
print("   - 1 outlier: age = 200 (impossible, should remove)")
print("   - 3 duplicates (should remove)")

# Simulate the decision-making process
print("\n" + "-" * 60)
print("DECISION 1: Income missing (15%)")
print("-" * 60)
print("   ❌ Can't remove: Would lose 15% of data (too much!)")
print("   ✅ Should impute: Use median (preserves distribution)")
print("   📝 Reason: Random missing, large percentage, numeric data")

print("\n" + "-" * 60)
print("DECISION 2: Email missing (2%)")
print("-" * 60)
print("   ✅ Can remove: Only 2% loss (acceptable)")
print("   ❌ Can't impute: Email is unique, can't estimate")
print("   📝 Reason: Small percentage, critical field, can't fill")

print("\n" + "-" * 60)
print("DECISION 3: Age outlier (age = 200)")
print("-" * 60)
print("   ✅ Should remove: Impossible value (data entry error)")
print("   ❌ Can't keep: Would break age-based models")
print("   📝 Reason: No human lives to 200, clearly an error")

print("\n" + "-" * 60)
print("DECISION 4: Duplicates (3 rows)")
print("-" * 60)
print("   ✅ Should remove: Exact duplicates, no new information")
print("   📝 Reason: Wastes space, can bias models (same data counted twice)")

print("\n" + "=" * 60)
print("✅ Decision Framework Applied Successfully!")
print("تم تطبيق إطار القرار بنجاح!")
print("=" * 60)


In [20]:
# Convert age to int

# df_clean['age'].round().astype(int)
# - df_clean['age']: Selects 'age' column
# - .round(): Rounds decimal values (e.g., 40.5 → 41)
# - .astype(int): Converts to integer type
# Result: Converts float ages to integer ages
df_clean['age'] = df_clean['age'].round().astype(int)
print("\nData types after conversion:")
print("أنواع البيانات بعد التحويل:")

# df_clean.dtypes
# - Shows data types after conversion
# - Should show int64 for age and experience_years now
print(df_clean.dtypes)